# Sensors measures

## 0. Clean Data

## 1. Read Files

In [14]:
import pandas as pd
import os

csv_files = list()
json_files = list()
rel_path="../data/raw/"
for root, dirs, files in os.walk(rel_path):
    for filename in files:
        if filename.split('.')[1] == 'csv':
            csv_files.append(pd.read_csv(rel_path + filename, parse_dates =["time"], index_col ="time"))
        else:
            json_files.append(pd.read_json(rel_path + filename,lines=True,convert_dates=['time']))
print(csv_files)
print(json_files)

[                            sensor  value
time                                     
2017-12-22 11:22:11+00:00  T-DHT11     27
2017-12-22 11:22:16+00:00  T-DHT11     28
2017-12-22 11:22:20+00:00  T-DHT11     28
2017-12-22 11:22:24+00:00  T-DHT11     28
2017-12-22 11:22:28+00:00  T-DHT11     28
...                            ...    ...
2018-03-13 11:41:53+00:00  T-DHT11     26
2018-03-13 11:41:57+00:00  T-DHT11     26
2018-03-13 11:42:01+00:00  T-DHT11     26
2018-03-13 11:42:05+00:00  T-DHT11     26
2018-03-13 11:42:09+00:00  T-DHT11     26

[1420308 rows x 2 columns],                             sensor  value
time                                     
2017-12-20 12:17:05+00:00  H-HTU21   28.3
2017-12-20 12:17:07+00:00  H-HTU21   28.2
2017-12-20 12:17:09+00:00  H-HTU21   28.1
2017-12-20 12:17:11+00:00  H-HTU21   28.2
2017-12-20 12:17:13+00:00  H-HTU21   28.2
...                            ...    ...
2018-03-13 11:41:55+00:00  H-HTU21   30.8
2018-03-13 11:41:59+00:00  H-HTU21   30.8
2018